<a href="https://colab.research.google.com/github/stfc/janus-tutorials/blob/main/geom_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geometry Optimization with machine learnt interatomic potentials

### Setup environment (optional)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!python3 -m pip install -U git+https://github.com/stfc/janus-core.git
!python3 -m pip install -U git+https://gitlab.com/ase/ase.git
!python3 -m pip install matgl
!python3 -m pip install chgnet

In [ ]:
from ase.io import read
from ase.visualize import view

from janus_core.calculations.single_point import SinglePoint
from janus_core.calculations.geom_opt import optimize

### Prepare for optimization of a deformed salt structure

In [ ]:
NaCl = read("data/NaCl-deformed.cif")

In [ ]:
view(NaCl)

In [ ]:
sp_mace = SinglePoint(
    struct=NaCl.copy(),
    architecture="mace_mp",
    device='cpu',
    calc_kwargs={'model_paths':'small','default_dtype':'float64'}
)

init_energy = sp_mace.run("energy")["energy"]

To optimize only the atomic positions and not the cell, set `filter_func = None`:

In [ ]:
optimized_NaCl = optimize(
    struct=sp_mace.struct,
    fmax=0.001,
    filter_func=None,
)

In [ ]:
view(optimized_NaCl)

Check energy has been lowered, and cell is unchanged:

In [ ]:
print(f"Initial cell: {NaCl.cell.cellpar()}")
print(f"Initial energy: {init_energy}")

print(f"Final cell: {optimized_NaCl.cell.cellpar()}")
print(f"Final energy: {optimized_NaCl.get_potential_energy()}")

### Optimizing cell vectors and atomic positions

Setting `filter_kwargs = {"hydrostatic_strain": True}` allows the cell lengths to be changed, in addition to atomic positions, but cell angles remain fixed:

In [ ]:
sp_mace_lengths = SinglePoint(
    struct=NaCl.copy(),
    architecture="mace_mp",
    device='cpu',
    calc_kwargs={'model_paths':'small','default_dtype':'float64'}
)

optimized_NaCl_lengths = optimize(
    struct=sp_mace_lengths.struct,
    fmax=0.001,
    filter_kwargs={"hydrostatic_strain": True},
)

Check energy has been lowered, and cell lengths have been updated, but angles remain unchanged:

In [ ]:
print(f"Initial cell: {NaCl.cell.cellpar()}")
print(f"Initial energy: {init_energy}")

print(f"Final cell: {optimized_NaCl_lengths.cell.cellpar()}")
print(f"Final energy: {optimized_NaCl_lengths.get_potential_energy()}")

### Optimizing at constant pressure and volume

Calculations can also be run at a fixed pressure and volume, by setting `filter_kwargs = {"scalar_pressure": x, "constant_volume": True}`

By default, both the cell lengths and angles will be optimized, in addition to the atomic positions.

We can also set the optimizer function and filter function used, either by passing the function itself (e.g. `FIRE`) or passing the name of the ASE function (e.g. `"ExpCellFilter"`):

In [ ]:
from ase.optimize import FIRE

sp_mace_pressure = SinglePoint(
    struct=NaCl.copy(),
    architecture="mace_mp",
    device='cpu',
    calc_kwargs={'model_paths':'small','default_dtype':'float64'}
)

optimized_NaCl_pressure = optimize(
    struct=sp_mace_pressure.struct,
    fmax=0.01,
    filter_kwargs={"scalar_pressure": 0.05, "constant_volume": True},
    optimizer=FIRE,
    filter_func="ExpCellFilter",
)

Check cell lengths and angles have both been updated:

In [ ]:
print(f"Initial cell: {NaCl.cell.cellpar()}")
print(f"Initial energy: {init_energy}")

print(f"Final cell: {optimized_NaCl_pressure.cell.cellpar()}")
print(f"Final energy: {optimized_NaCl_pressure.get_potential_energy()}")

## Comparing MACE to CHGNet and M3GNet

In [ ]:
sp_mace = SinglePoint(
    struct=NaCl.copy(),
    architecture="mace_mp",
    device='cpu',
    calc_kwargs={'model_paths':'small','default_dtype':'float64'}
)

optimized_NaCl_mace = optimize(
    struct=sp_mace.struct,
    fmax=0.01,
)

sp_chgnet = SinglePoint(
    struct=NaCl.copy(),
    architecture="chgnet",
    device="cpu"
)


optimized_NaCl_chgnet = optimize(
    struct=sp_chgnet.struct,
    fmax=0.01,
)

sp_m3gnet = SinglePoint(
    struct=NaCl.copy(),
    architecture="m3gnet",
    device="cpu"
)


optimized_NaCl_m3gnet = optimize(
    struct=sp_m3gnet.struct,
    fmax=0.01,
)

In [ ]:
print(f"Initial energy: {init_energy}")


print(f"Final energy (MACE): {optimized_NaCl_mace.get_potential_energy()}")
print(f"Final energy (CHGNET): {optimized_NaCl_chgnet.get_potential_energy()}")
print(f"Final energy (M3GNET): {optimized_NaCl_m3gnet.get_potential_energy()}")